                                            Import libraries

In [1]:
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

                                            Import data set

In [71]:
#import images path from local disk
train_dir = 'data/train'
val_dir = 'data/test'
#Generate batches of tensor image data with real-time data augmentation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
#split data to train and validation
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
    )

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
   )


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


Model Build

In [72]:
model = Sequential()
model.add(Conv2D(64,kernel_size =(3,3),strides=1,activation='relu',input_shape=(48,48,1)))#2D convolution layer
model.add(Conv2D(64,kernel_size =(3,3),strides=1,activation='relu',input_shape=(48,48,64)))#2D convolution layer
model.add(MaxPooling2D(pool_size=(2,2),strides=2,input_shape=(46,46,64)))#Max Pooling
model.add(Dropout(0.25))#Dropout is a technique where randomly selected neurons are ignored during training.
model.add(Conv2D(128, kernel_size=(3,3),strides=1, activation='relu',input_shape=(23,23,64)))#2D convolution layer
model.add(Conv2D(128, kernel_size=(3,3), strides=1, activation='relu',input_shape=(23,23,128)))#2D convolution layer
model.add(Conv2D(256, kernel_size=(3,3), strides=1, activation='relu',input_shape=(21,21,128)))#2D convolution layer
model.add(Conv2D(256, kernel_size=(3,3), strides=1, activation='relu',input_shape=(21,21,256)))#2D convolution layer
model.add(MaxPooling2D(pool_size=(2,2),strides=2,input_shape=(19,19,256)))#Max Pooling
model.add(MaxPooling2D(pool_size=(2,2),strides=2,input_shape=(9,9,256)))#Max Pooling
model.add(Dropout(0.25))#Dropout is a technique where randomly selected neurons are ignored during training.
model.add(Flatten())#Flatten is used to flatten the input.
model.add(Dense(1024, activation='relu'))#Dense layer feeds all outputs from the previous layer to all its neurons
model.add(Dropout(0.5))#Dropout layer
model.add(Dense(7, activation='softmax'))#Dense layer feeds all outputs from the previous layer to all its neurons

Model training

In [74]:
checkpoint_path = "training_1/cp.ckpt"#create a checkpoint folder
checkpoint_dir = os.path.dirname(checkpoint_path)#get path of checkpoint folder
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)#save model weight fo each epoch

In [76]:
#Model compile
model.compile(loss='categorical_crossentropy' , optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
#train model
modelTrain = model.fit(train_generator,steps_per_epoch= 64 ,
                                 epochs=100,
                                 validation_data=validation_generator,
                                 validation_steps= 64,
                                callbacks=[cp_callback])
#save model
model.save('face_model')
os.listdir(checkpoint_dir)#check save weight in check point folder

Epoch 1/100
64/64 [==============================] - ETA: 0s - loss: 1.8226 - accuracy: 0.2497
Epoch 1: saving model to training_1\cp.ckpt
64/64 [==============================] - 102s 2s/step - loss: 1.8226 - accuracy: 0.2497 - val_loss: 1.8103 - val_accuracy: 0.2532
Epoch 2/100
64/64 [==============================] - ETA: 0s - loss: 1.7980 - accuracy: 0.2627
Epoch 2: saving model to training_1\cp.ckpt
64/64 [==============================] - 99s 2s/step - loss: 1.7980 - accuracy: 0.2627 - val_loss: 1.7991 - val_accuracy: 0.2480
Epoch 3/100
64/64 [==============================] - ETA: 0s - loss: 1.7783 - accuracy: 0.2700
Epoch 3: saving model to training_1\cp.ckpt
64/64 [==============================] - 93s 1s/step - loss: 1.7783 - accuracy: 0.2700 - val_loss: 1.7797 - val_accuracy: 0.2637
Epoch 4/100
64/64 [==============================] - ETA: 0s - loss: 1.7809 - accuracy: 0.2632
Epoch 4: saving model to training_1\cp.ckpt
64/64 [==============================] - 92s 1s/step - l

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

Model Training

In [77]:
model.summary()#model summary

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 46, 46, 64)        640       
                                                                 
 conv2d_41 (Conv2D)          (None, 44, 44, 64)        36928     
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 22, 22, 64)       0         
 g2D)                                                            
                                                                 
 dropout_21 (Dropout)        (None, 22, 22, 64)        0         
                                                                 
 conv2d_42 (Conv2D)          (None, 20, 20, 128)       73856     
                                                                 
 conv2d_43 (Conv2D)          (None, 18, 18, 128)       147584    
                                                      

Get output from model